# Face recognition

### Список необходимых библиотек:

In [56]:
import numpy as np
import os
import math
from PIL import Image
from PIL import ImageDraw

### Глобальные переменные:

In [57]:
widthOfImages = 0
heightOfImages = 0
numberOfRows = 0
numberOfColumns = 0
folder = "att_faces"

### Обработать данные, перевести их в матрицу m*n:

    Метод возвращающий список всех путей фотографий из заданной папки

In [58]:
def readPathesOfImagesFromFolder ():
    listOfSubfolders = os.listdir(folder)
    listOfImagesPathes = list()
    listOfLabels = list()
    label = 0
    for i in listOfSubfolders:
        listOfImagesInCurrentSubfolder = os.listdir(folder+"/"+i)
        label += 1
        for j in listOfImagesInCurrentSubfolder:            
            listOfImagesPathes.append(folder+"/"+i+"/"+j)
            listOfLabels.append(label)
    np.savetxt('listOfLabels.txt', listOfLabels)
    return listOfImagesPathes, listOfLabels

    Метод заполняющий глобальные переменные widthOfImages, heightOfImages, numberOfRows, numberOfColumns:

In [59]:
def countGlobalVariable():
    pathesOfImages, listOfLabels = readPathesOfImagesFromFolder()
    firstImage = Image.open(pathesOfImages[0])
    global  widthOfImages
    global  heightOfImages
    global  numberOfRows
    global  numberOfColumns
    widthOfImages = firstImage.size[0]    
    heightOfImages = firstImage.size[1]
    numberOfRows = len(pathesOfImages)
    numberOfColumns = widthOfImages*heightOfImages

Перевод списка путей в попиксельную матрицу m*n:

In [60]:
def convertPathesToImagesMatrix (listOfImagesPathes):
    countGlobalVariable()
    global numberOfRows
    global numberOfColumns
    matrixOfImagesPixels = np.zeros((numberOfRows,numberOfColumns))
    currentRow = 0
    for f in listOfImagesPathes:
        im=np.array(Image.open(f))
        im=im.reshape(1,im.shape[0]*im.shape[1])
   
        matrixOfImagesPixels[currentRow,:]=im/255
        currentRow+=1
    np.savetxt('matrixOfImagesPixels.txt', matrixOfImagesPixels)
    return matrixOfImagesPixels


### Разделяет матрицу изоражений на эталонную и тестовую с заданным процентом тестовых изобрашений:

In [61]:
def separateMatrixToTrainAndTestMatrix (matrixOfImagesPixels, listOfLabels, countTestPersent):
    testMatrix = np.zeros(((numberOfRows*countTestPersent)//100,numberOfColumns))
    i_testMatrix = 0
    testLabels = list()
    trainMatrix = np.zeros((numberOfRows - testMatrix.shape[0],numberOfColumns))
    i_trainMatrix = 0
    trainLabels = list()
    
    for i in range(0, numberOfRows):
        if (i%10<countTestPersent//10):
            testMatrix[i_testMatrix,:]=matrixOfImagesPixels[i,:]
            i_testMatrix+=1
            testLabels.append(listOfLabels[i])            
        else:
            trainMatrix[i_trainMatrix,:]=matrixOfImagesPixels[i,:]
            i_trainMatrix+=1
            trainLabels.append(listOfLabels[i])
    return trainMatrix, trainLabels, testMatrix, testLabels

### Функция подсчета евклидового расстояния между фотографиями:
На вход принимает две фотографии, возвращает одно число - евклидово расстояние

In [62]:
def evklidDistanse(vector01, vector02):
    d = np.linalg.norm(vector01-vector02)
    return d

### Aлгоритм распознавания:

In [63]:
def discoverPerson(person, label, matrixOfImagesPixels, listOfLabels):
    minDis = evklidDistanse(person, matrixOfImagesPixels[0])
    currentPersonLabel = 1
    for i in range(1,len(matrixOfImagesPixels)):
        d = evklidDistanse(person, matrixOfImagesPixels[i])
        if (d < minDis):
            minDis = d
            currentPersonLabel = listOfLabels[i]
    return currentPersonLabel, currentPersonLabel ==label

### Нормализация:

Входной параметр:
+ 0 - без нормализации
+ 1 - стандартное отклонение по семплам
+ 2 - стандартное отклонение по фичам

In [64]:
def normalizeStandardDeviation(matrixOfImagesPixels, mode):
    if (mode==0):
        matrixOfImagesPixels = matrixOfImagesPixels
    elif(mode==1):
        for i in range(0, len(matrixOfImagesPixels)):
            matrixOfImagesPixels[i] = (matrixOfImagesPixels[i] - np.mean(matrixOfImagesPixels[i]))/np.std(matrixOfImagesPixels[i])
    else:   
        for i in range(0, len(matrixOfImagesPixels[0])):
            matrixOfImagesPixels[:,i] = (matrixOfImagesPixels[:,i] - np.mean(matrixOfImagesPixels[:,i]))/np.std(matrixOfImagesPixels[:,i])
    return matrixOfImagesPixels

### Алгоритм генерации и записи в файл конвертациинной матрицы Е по заданной тестовой выборке, РСА

In [65]:
def generatePCA(countTestPercents, PCA, matrix):
    nameOfMatrixFile = "E_"+str(countTestPercents)+"_"+"0,"+str(PCA)+".txt"
    
    #считаем ковариационную матрицу
    covMatrix = np.dot(matrix.transpose(),matrix)
    
    #считаем собственные вектора
    eigenvalues, eigenvectors  = np.linalg.eig(covMatrix)
    
    #сортируем собственные числа и вектора
    for i in range(0,len(eigenvalues)):
        for j in range(1,len(eigenvalues)):
            if (eigenvalues[j]>eigenvalues[j-1]):
                temp = eigenvalues[j]
                eigenvalues[j] = eigenvalues[j-1]
                eigenvalues[j-1] = temp
            
                temp = eigenvectors[j]
                eigenvectors[j] = eigenvectors[j-1]
                eigenvectors[j-1] = temp
    #считаем К
    K = 1
    
    #подсчет суммы
    summ = 0
    for i in eigenvalues:
        summ = summ + i
    
    #подсчет нужного К
    currentSum = 0
    i = 0
    while (currentSum/summ<PCA/100):
        currentSum = currentSum + eigenvalues[i]
        i = i + 1
        K = K + 1
    
    #получаем матрицу перехода
    E = np.zeros((K, len(matrix[0])))
    for i in range(0,K):
        E[i] = eigenvectors[i]
    
    #cохраняем текущую матрицу РСА
    np.savetxt(nameOfMatrixFile, E)
    print(nameOfMatrixFile)

### PCA (по готовой матрице Е из текстового файла перехода для выборки заданного размера):

In [66]:
def convertPCA(countTestPercents, PCA, matrix):
    if (PCA != 0):
        nameOfMatrixFile = "E_"+str(countTestPercents)+"_"+"0,"+str(PCA)+".txt"
        E = np.loadtxt(nameOfMatrixFile)
        E = E.transpose()
        matrix = np.dot(matrix, E)
    return matrix

### Тестирование с заданными параметрами(процент тестовой выборки, нормализация, PCA):

In [67]:
def testProgram(countTestPercents, normalize, PCA):
    listOfImagesPathes, listOfLabels = readPathesOfImagesFromFolder()
    matrixOfImagesPixels = convertPathesToImagesMatrix (listOfImagesPathes)
    
    #нормализация
    matrixOfImagesPixels = normalizeStandardDeviation(matrixOfImagesPixels, normalize)
    
    #разделение на эталонную и тестовую выборки
    trainMatrix, trainLabels, testMatrix, testLabels = separateMatrixToTrainAndTestMatrix(matrixOfImagesPixels, listOfLabels, countTestPercents)
    
    #РСА
    trainMatrix = convertPCA(countTestPercents, PCA, trainMatrix)
    testMatrix = convertPCA(countTestPercents, PCA, testMatrix)
    print("K = "+str(len(testMatrix[0])))
    
    corrects = 0
    for i in range(0, len(testMatrix)):
        n, b = discoverPerson(testMatrix[i], testLabels[i], trainMatrix, trainLabels)
        if b:
            corrects+=1
    corrects = (corrects/len(testMatrix))*100
    return corrects


### Вывод:

In [68]:
print("При разделении 90/10 без нормализации без PCA эффективность:" + str(testProgram(10, 0, 0)) + "%")
print("При разделении 80/20 без нормализации без PCA эффективность:" + str(testProgram(20, 0, 0)) + "%")
print("При разделении 70/30 без нормализации без PCA эффективность:" + str(testProgram(30, 0, 0)) + "%")
print("При разделении 60/40 без нормализации без PCA эффективность:" + str(testProgram(40, 0, 0)) + "%")
print("При разделении 50/50 без нормализации без PCA эффективность:" + str(testProgram(50, 0, 0)) + "%")
print()

print("При разделении 90/10 нормализация по семплам без PCA эффективность:" + str(testProgram(10, 1, 0)) + "%")
print("При разделении 80/20 нормализация по семплам без PCA эффективность:" + str(testProgram(20, 1, 0)) + "%")
print("При разделении 70/30 нормализация по семплам без PCA эффективность:" + str(testProgram(30, 1, 0)) + "%")
print("При разделении 60/40 нормализация по семплам без PCA эффективность:" + str(testProgram(40, 1, 0)) + "%")
print("При разделении 50/50 нормализация по семплам без PCA эффективность:" + str(testProgram(50, 1, 0)) + "%")
print()

print("При разделении 90/10 нормализация по фичам без PCA эффективность:" + str(testProgram(10, 2, 0)) + "%")
print("При разделении 80/20 нормализация по фичам без PCA эффективность:" + str(testProgram(20, 2, 0)) + "%")
print("При разделении 70/30 нормализация по фичам без PCA эффективность:" + str(testProgram(30, 2, 0)) + "%")
print("При разделении 60/40 нормализация по фичам без PCA эффективность:" + str(testProgram(40, 2, 0)) + "%")
print("При разделении 50/50 нормализация по фичам без PCA эффективность:" + str(testProgram(50, 2, 0)) + "%")
print()

print("При разделении 80/20 без нормализации c PCA=95 эффективность:" + str(testProgram(20, 0, 95)) + "%")
print("При разделении 80/20 без нормализации c PCA=98 эффективность:" + str(testProgram(20, 0, 98)) + "%")
print("При разделении 80/20 без нормализации c PCA=99 эффективность:" + str(testProgram(20, 0, 99)) + "%")

K = 10304
При разделении 90/10 без нормализации без PCA эффективность:97.5%
K = 10304
При разделении 80/20 без нормализации без PCA эффективность:96.25%
K = 10304
При разделении 70/30 без нормализации без PCA эффективность:97.5%
K = 10304
При разделении 60/40 без нормализации без PCA эффективность:94.375%
K = 10304
При разделении 50/50 без нормализации без PCA эффективность:93.5%

K = 10304
При разделении 90/10 нормализация по семплам без PCA эффективность:95.0%
K = 10304
При разделении 80/20 нормализация по семплам без PCA эффективность:92.5%
K = 10304
При разделении 70/30 нормализация по семплам без PCA эффективность:95.0%
K = 10304
При разделении 60/40 нормализация по семплам без PCA эффективность:92.5%
K = 10304
При разделении 50/50 нормализация по семплам без PCA эффективность:91.0%

K = 10304
При разделении 90/10 нормализация по фичам без PCA эффективность:97.5%
K = 10304
При разделении 80/20 нормализация по фичам без PCA эффективность:96.25%
K = 10304
При разделении 70/30 нормал